### Welcome to the second exercise on E-ID in this E-ID Workshop
Before we start, make sure you have everything installed by following one of the outlined methods in the ReadMe file. 

### Purpose of this exercise
This jupyter notebook explains the following:
1. Basic E-ID example using the BBS cryptographic scheme
2. Selective Disclosure using BBS
3. Unlinkability using BBS and what role Zero-knowledge proofs play in it.
5. The previous points will use the **docknetwork crypto library**

The author of this crypto library is Dock.io

Dock.io is one of the leading startups in the Crypto E-ID space.  
They open-source many fundamental libraries including the one we're using it today.
- Dock.io has 16 employess published on their website
- Originally Swiss but based in the US
- Some 20 million USD in funding (Web3 Foundation - lead investor)

---
## Technical overview
In this exercise, we will explore the BBS cryptographic scheme, and see how it differs from a classic RSA scheme as explored in the previous exercise.

For this, we will use a different typescript library https://github.com/docknetwork/crypto-wasm-ts


In [12]:
// We start by declaring our most important asset which is the E-ID Verifiable Credential we will be working with.

const E_ID_DATA = {
    name: "Jack Sparrow",
    timeOfBirth: 744156000000,
    profession: "IT Manager"
}

// For this advanced library, we will also need to specify the schema of Credential we're working with.
// Specifying the schema will be useful later to get the correct encoding of each field.
const E_ID_SCHEMA = {
    type: 'object',
    properties: {
        name: { type: 'string' },
        profession: { type: 'string' },
        timeOfBirth: { type: 'integer', minimum: 0},
    }
}

### JSON Schema
The schema above is defined using JSON schema.  
JSON Schema is the standard for defining the structure/ schema of a JSON object. It's typically used
to define a contract for APIs and data communication (which use JSON).  
One known standard that builds on JSON schema is the Open API Specification (OAS - previously known as swagger). 
OAS -in its latest versions- uses a superset of the latest published [JSON schema](https://json-schema.org/specification) (2020-12)

In [13]:
import { initializeWasm } from '@docknetwork/crypto-wasm-ts'
await initializeWasm();

## Issuer

In [14]:
// Generating a keypair once signature parameters are created.
import { BBSKeypair, BBSSignatureParams, BBS_SIGNATURE_PARAMS_LABEL_BYTES } from '@docknetwork/crypto-wasm-ts';

const stringToBytes = (str: string) => Uint8Array.from(Buffer.from(str, "utf-8"));

const params = BBSSignatureParams.generate(100, BBS_SIGNATURE_PARAMS_LABEL_BYTES);
const keypair = BBSKeypair.generate(params, stringToBytes('seed1'));

const secretKey = keypair.secretKey;
const publicKey = keypair.publicKey;

In [15]:
// Issing a credential based on the E-ID data based on the specified schema
import { CredentialSchema, BBSCredentialBuilder, BBSCredential, SUBJECT_STR } from '@docknetwork/crypto-wasm-ts'


// sets a default structure for a credential with the proof 
let credential_schema = CredentialSchema.essential();
// Puts our data schema inside the credential schema
credential_schema.properties[SUBJECT_STR] = E_ID_SCHEMA


// Populate the credential with data
const builder = new BBSCredentialBuilder();
builder.schema = new CredentialSchema(credential_schema);
builder.subject = E_ID_DATA

// sign the credential
const verifiableCredential = builder.sign(secretKey);

// 
verifiableCredential.verify(publicKey)

{ verified: true, error: undefined }


In [18]:
// Now, the credential should be sent to the holder through some kind of medium!
const credentialToSendToHolder = verifiableCredential.toJSON()
credentialToSendToHolder

{
  cryptoVersion: '0.7.0',
  credentialSchema: {
    id: 'data:application/json;charset=utf-8,',
    type: 'JsonSchemaValidator2018',
    version: '0.5.0',
    details: '{"jsonSchema":{"$schema":"http://json-schema.org/draft-07/schema#","definitions":{"encryptableCompString":{"type":"string"},"encryptableString":{"type":"string"}},"properties":{"credentialSubject":{"properties":{"name":{"type":"string"},"profession":{"type":"string"},"timeOfBirth":{"minimum":0,"type":"integer"}},"type":"object"},"proof":{"properties":{"type":{"type":"string"}},"type":"object"}},"type":"object"},"parsingOptions":{"defaultDecimalPlaces":0,"defaultMinimumDate":-17592186044415,"defaultMinimumInteger":-4294967295,"useDefaults":false}}'
  },
  credentialSubject: {
    name: 'Jack Sparrow',
    timeOfBirth: 744156000000,
    profession: 'IT Manager'
  },
  proof: {
    type: 'Bls12381BBSSignatureDock2023',
    proofValue: '2vLcFWoLoFweEQNwTwETf4Xtyc3mK1DwDX4RZyDuYsxSNtMzGvAMjmpXZgUPyKCRimAyBbNjqo99JotuDczkkb

## Credential Holder

In [19]:
// The holder loads the credential back from JSON
const receivedCredential = BBSCredential.fromJSON(credentialToSendToHolder)
// console.log(receivedCredential)


import { PresentationBuilder } from '@docknetwork/crypto-wasm-ts'
const presentationBuilder = new PresentationBuilder();
presentationBuilder.addCredential(receivedCredential, publicKey)
presentationBuilder.markAttributesRevealed(0, new Set<string>(['credentialSubject.name', 'credentialSubject.profession']));

const presentation = presentationBuilder.finalize();
console.log(presentation)

Presentation {
  _version: '0.9.0',
  spec: PresentationSpecification {
    credentials: [ [Object] ],
    attributeEqualities: [],
    boundedPseudonyms: {},
    unboundedPseudonyms: {}
  },
  proof: CompositeProof {
    value: Uint8Array(427) [
        1,   0,   0,   0,   0,   0,   0,   0,  11, 160, 138, 199,
      251,  31,  22, 218, 192, 131, 219, 196, 212, 127, 253,  29,
      228, 149, 227,  98, 143,  83, 204,  14, 199, 184,  10,  95,
       73,  13, 239, 227,  23,  42,  57, 204,  20, 242, 151, 195,
       90,  10, 156, 140, 180, 127, 118, 197, 215, 146, 115,  55,
      194,  10,  94, 185,   1,  73,  58,  25,  92,  60,  76, 152,
       28, 128, 184, 221, 224, 194, 253, 106, 169,  77, 178, 189,
       75, 160, 117, 193,  37,  56, 187,  52,   9,  36, 145,  51,
      172, 239, 140, 155,
      ... 327 more items
    ]
  },
  attributeCiphertexts: undefined,
  blindedAttributeCiphertexts: undefined,
  context: undefined,
  nonce: undefined
}


### sending data to Verifier
data: 
1. selected data to be revealed
2. Schema of the credential
3. Proof of validity

All these info and more are stored in the "presentation object". Note that this object is a completely serializable JSON object.

## Verifier

In [20]:
const receivedObjectOnVerifierSide = presentation.toJSON()
console.log(receivedObjectOnVerifierSide)  // Assume this was serialized and sent across an API to the verifier.
console.log("-------------------------")
console.log(presentation.spec.credentials[0])

{
  version: '0.9.0',
  context: undefined,
  nonce: null,
  spec: {
    credentials: [ [Object] ],
    attributeEqualities: [],
    boundedPseudonyms: {},
    unboundedPseudonyms: {}
  },
  proof: 'PqJrdz8W5DWEPF9a8edj1xVcWxKQpq7TUeL9NZ7Pdk4aoqXXZ8SmsAZJfpkidXxAufheN3GHHNRmdbSA4Wt27NMMHYhXtf4aX114yE1mY5d18mJE3XSkBx1U9prUPA83xrJpykLSdwMHYpUHLZ2Yy51pVzNHrNqnNwnBRP6Qh8wom1Zez97gqSyRapWs1t4f6G5J1GG8YjgBoVY5AWbkEqFhF1Qm95yy9VXWQebKSoZJB4isjudkwgsbbwvFoKHuVBAQaJgzEMGmEk2Jzi27m1GarLJ1eLq2rC7pi62M4gLEHAR6KR1bg9Y6JZedVwm4ET38c52TsZ5Vi3v9FqXTQp8owSqHTNoNBnWdSK2gUk9L6U7YXh1xdxbCmryW9KRnQaEwqaqZus6WNgYs8PQ9FCcwWuLdxMQd9PZ4Xg9nzDbBbLsfpRPoAtmKYAS9EeGhCniMg9SHmj2ALVgt76BpFYZ8JteYEd23fAGiJw6b4fbLT7uoSCK447VjG3zTxcN1o1WSXDgA5Wi2PNtovpfuGJurjt6A8QYD4Lc4feqzbd97ip5KZDR5RVMgaqzsG8xe8a9Tyy'
}
-------------------------
{
  version: '0.7.0',
  schema: {
    details: '{"jsonSchema":{"$schema":"http://json-schema.org/draft-07/schema#","definitions":{"encryptableCompString":{"type":"string"},"encryptableStrin

In [21]:
// Verification of the data the verifier received is as easy as two lines of code!
import { Presentation } from '@docknetwork/crypto-wasm-ts'

const recreatedPres = Presentation.fromJSON(receivedObjectOnVerifierSide)
recreatedPres.verify([publicKey])

// Now, we can go ahead and access the shared data!
//recreatedPres.spec.credentials[0].revealedAttributes

{ verified: true, error: undefined }


### Is this cryptographic scheme unlinkable?

We can know that by checking for two conditions

1. Is there a signature or a cryptopgraphic data that is shared and could used to identify the credential holder?
2. Is there a part of data hashed or not that was included in the presented data that could be used to identify the credential holder?

In [22]:
presentationBuilder.finalize()["proof"].hex

01000000000000000bad7266872b1d321552ac0d014fde245522decae9c6b914cfada8cdaf4c525c19b48ac710b97d9a650f64f3b5cb6fd21ca3c739cfcdbdf26e3c06cbd331d51777d1d39fb6b6f15e7647639716a08649b5d4e1407458dfa4a937b54030453be74b846c6ed3a95eb98268ca71d1ce33d80751f78f85b0a50515f6b7d3acbcf1ced91d60e0ca2f438dcbc894808cc7ab8c9786f84ecdfe70bfa65947807251d971d402e38d85073043863efe7198fb2a7e89ad13fc92ef51693494b8ec2df800e1e18eb3a6c08d4ecc0b74dc344c7402b71f5d8062b8aa12e1defbcd6ecdae129a43c8a7056a3e87f7b8dd0e34cb20251cb285759d624b7df5f836758acef16f5248b7395fd66a9e0c5a10408a688177e877d05fc77c0bf82b81b1364ae09d17d9dca44be3e08fdebf43423dd3a8b82ab7a103000000000000001f418c0fd1eaee43f335e17f0c5deaf993625817feb7c6ef27281cb0cc1c7559e3358134422588677327c2e05bf0f71e713ce1b9b556472c4a8e3e44e43f883b355d3fb4e18b7f810c04b379735d6c3f8687b178965be0addcf8e5516f3e243800000000000000000000


In [23]:
// To answer the first point, we look at the presented proof 
const proof1 = presentationBuilder.finalize()["proof"].hex
const proof2 = presentationBuilder.finalize()["proof"].hex

// By generating a proof multiple times, and comparing the proof in each of them,
// we see that they're always different making it difficult (Impossible?!) to correlate the different proofs.
console.log(proof1 === proof2)

// This is a weak proof! In the paper, there's a real mathematical proof to 
// ensure that different generated proofs are unlinkable 

false


In [24]:
// To answer the second point, we look at the data shared with the verifier.
// As opposed to the previous exercise where we had to share all the data all the time
// even though possibly in a hashed format, in this case, we only share the selected data.

presentation.spec.credentials[1]["revealedAttributes"]

// In this scenario, we see that only the data needed to be revealed is revealed resulting in unlinkability.

{
  credentialSubject: { name: 'Jack Sparrow', profession: 'IT Manager' }
}


## Exercise: 
Divide into three teams (issuers (min:1), holders(min:2), verifiers(min:1))
and use a chat tool to communicate. 
-> Linus already used a chat tool in his classes!

-- (Maybe) do a small PKI somewhere.